# 1次元の畳み込みニューラルネットワークスクラッチ

## 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成
---
チャンネル数を1に限定した1次元畳み込み層のクラスSimpleConv1dを作成してください。基本構造は前のSprintで作成した全結合層のFCクラスと同じになります。なお、重みの初期化に関するクラスは必要に応じて作り変えてください。Xavierの初期値などを使う点は全結合層と同様です。

ここでは*パディング*は考えず*ストライド*も1に固定します。また、複数のデータを同時に処理することも考えなくて良く、バッチサイズは1のみに対応してください。この部分の拡張はアドバンス課題とします。

フォワードプロパゲーションの数式は以下のようになります。

$$
a_i = \sum_{s=0}^{F-1}x_{(i+s)}w_{s+b}
$$

$a_i$: 出力される配列のi番目の値

$F$: フィルタのサイズ

$x_{(i+s)}$: 入力の配列の(i+s)番目の値

$w_{s}$: 重みの配列のs番目の値

$b$: バイアス項

すべてスカラーです。

次に更新式です。ここがAdaGradなどに置き換えられる点は全結合層と同様です。
$$
w_s^{\prime} = w_s - \alpha \frac{\partial L}{\partial w_s} \\
b^{\prime} = b - \alpha \frac{\partial L}{\partial b}
$$

α: 学習率

$\frac{\partial L}{\partial a_i}$: ws に関する損失$L$の勾配

$\frac{\partial L}{\partial b}$: b に関する損失$L$の勾配

勾配$\frac{\partial L}{\partial w_s}$や$\frac{\partial L}{\partial b}$を求めるためのバックプロパゲーションの数式が以下です。
$$
\frac{\partial L}{\partial w_s} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}x_{(i+s)}\\
\frac{\partial L}{\partial b} = \sum_{i=0}^{N_{out}-1} \frac{\partial L}{\partial a_i}
$$

$\frac{\partial L}{\partial a_i}$: 勾配の配列のi番目の値

$N_{out}$: 出力のサイズ

前の層に流す誤差の数式は以下です。

$$\frac{\partial L}{\partial x_j} = \sum_{s=0}^{F-1} \frac{\partial L}{\partial a_{(j-s)}}w_s$$

$\frac{\partial L}{\partial x_j}$: 前の層に流す誤差の配列のj番目の値

ただし、$j−s<0$または$j−s>N_{out}−1$のとき$\frac{\partial L}{\partial a_{(j-s)}} =0$です。

全結合層との大きな違いは、重みが複数の特徴量に対して共有されていることです。この場合は共有されている分の誤差をすべて足すことで勾配を求めます。計算グラフ上での分岐はバックプロパゲーションの際に誤差の足し算をすれば良いことになります。

In [34]:
import numpy as np

class SimpleConv1d:
    """
    畳み込み層
    Parameters
    ----------
    W : ndarray
      重み
    B : ndarray
      バイアス
    """
    def __init__(self, W, B):
        self.stride = 1     #今回は1固定
        self.padding = 0    #今回は無し
        self.batch_size = 1 # #今回は1固定

        self.W = W
        self.B = B

        # 微分した重みとバイアス
        self.dW = None
        self.dB = None

        self.x = None
        self.x_matrix = None
        self.dZ = None

    def _forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """

        self.x = X  # backwardで使うため保持

        idx = 0
        # n_row = len(X) - len(self.W) + self.stride
        n_output = self._get_output_size()
        x_list = []
        for i in range(n_output):
            # print(X[idx:idx + len(self.W)])
            x_list.append(X[idx:idx + len(self.W)])
            idx += self.stride

        self.x_matrix = np.array(x_list)

        A = np.sum(self.W.transpose() * np.array(x_list), axis=1) + self.B

        return A

    def _backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """

        # n_row = len(self.x) - len(self.W) + self.stride
        n_output = self._get_output_size()
        idx = diff = len(self.x) - len(self.W)
        w_list = []
        for i in range(n_output):
            _w = np.pad(self.W, [idx-diff, diff],'constant')
            w_list.append(_w)
            diff -= self.stride

        dx = np.sum(dA.reshape(-1,1) * np.array(w_list), axis=0)
        self.dW = np.sum(self.x_matrix.T * dA, axis=1)
        self.dB = np.array(dA.sum())

        return dx
    
    #問題2
    def _get_output_size(self):
        '''
        出力サイズの計算 n_output = (n_input + 2*padding - n_filters)/stride + 1
        '''

        return int((len(self.x) + 2*self.padding - len(self.W))/ self.stride + 1)


## 【問題2】1次元畳み込み後の出力サイズの計算
---
畳み込みを行うと特徴量の数が変化します。どのように変化するかは以下の数式から求められます。パディングやストライドも含めています。この計算を行う関数を作成してください。
$$
N_{out} =  \frac{N_{in}+2P-F}{S} + 1
$$
$N_{out}$: 出力のサイズ（特徴量の数）

$N_{in}$: 入力のサイズ（特徴量の数）

$P$: ある方向へのパディングの数

$F$: フィルタのサイズ

$S$: ストライドのサイズ

## 【問題3】小さな配列での1次元畳み込み層の実験
---
次に示す小さな配列でフォワードプロパゲーションとバックプロパゲーションが正しく行えているか確認してください。

入力x、重みw、バイアスbを次のようにします。

```python
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])    
```

フォワードプロパゲーションをすると出力は次のようになります。
```python
a = np.array([35, 50])
```
次にバックプロパゲーションを考えます。誤差は次のようであったとします。
```python
delta_a = np.array([10, 20])
```
バックプロパゲーションをすると次のような値になります。
```python
delta_b = np.array([30])
delta_w = np.array([50, 80, 110])
delta_x = np.array([30, 110, 170, 140])
```

In [35]:
 x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])

conv1d = SimpleConv1d(w, b)
a = conv1d._forward(x)
print("a : ", a)

delta_a = np.array([10, 20])

dx = conv1d._backward(delta_a)
print("dB : ", conv1d.dB)
print("dW : ", conv1d.dW)
print("dx : ", dx)

a :  [35 50]
dB :  30
dW :  [ 50  80 110]
dx :  [ 30 110 170 140]


## 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成
---
チャンネル数を1に限定しない1次元畳み込み層のクラスConv1dを作成してください。

例えば以下のようなx, w, bがあった場合は、
```python
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）
```
出力は次のようになります。
```python
a = np.array([[16, 22], [17, 23], [18, 24]]) # shape(3, 2)で、（出力チャンネル数、特徴量数）
```

入力が2チャンネル、出力が3チャンネルの例です。計算グラフを書いた上で、バックプロパゲーションも手計算で考えてみましょう。計算グラフの中には和と積しか登場しないので、微分を新たに考える必要はありません。

《補足》

チャンネル数を加える場合、配列をどういう順番にするかという問題があります。(バッチサイズ、チャンネル数、特徴量数)または(バッチサイズ、特徴量数、チャンネル数)が一般的で、ライブラリによって順番は異なっています。（切り替えて使用できるものもあります）

今回のスクラッチでは自身の実装上どちらが効率的かを考えて選んでください。上記の例ではバッチサイズは考えておらず、(チャンネル数、特徴量数)です。

In [36]:
class Conv1d:
    """
    1D畳み込み層
    Parameters
    ----------
    W : ndarray
      重み
    B : ndarray
      バイアス
    """
    def __init__(self, W, B):
        self.stride = 1     #今回は1固定
        self.padding = 0    #今回は無し
        self.batch_size = 1 # #今回は1固定
        self.W = W
        self.B = B

        # 微分した重みとバイアス
        self.dW = None
        self.dB = None


        self.x = None
        self.x_matrix = None
        self.dZ = None

        # AdaGrad用のインスタンス変数
        self.W_h = 0
        self.B_h = 0

    def _forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """

        self.x = X  # backwardで使うため保持

        idx = 0
        n_output = self._get_output_size()
        n_channel = self.W.shape[0]
        n_filters = self.W.shape[2]
        A = np.zeros((n_channel, n_output))
        x_list = [] # backwardで使用する

        # 畳み込み演算用にXデータをフィルターと同じ形状に分割し、backwardで使用するため保持もする
        for _ in range(n_output):
            _X = X[:, idx:idx + n_filters]
            x_list.append(_X) #1次元配列に形状を変更（平滑化）
            # x_list.append(_X.reshape(1, -1)) #1次元配列に形状を変更（平滑化）
            idx += self.stride

        # backwardで使用するため保持する。
        self.x_matrix = np.array(x_list).reshape(n_output, -1)

        # 畳み込み演算
        for i in range(n_channel):
            for j, _X in enumerate(x_list):
                A[i][j] = (np.dot(_X.reshape(1, -1), self.W[i].reshape(1, -1).T)) + self.B[i]

        return A

    def _backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dX : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """

        n_output = self._get_output_size()
        n_channel, n_features = self.x.shape
        n_filters = self.W.shape[-1]
        idx = diff = n_features - n_filters

        w_list = []
        for i in range(n_filters):
            for _ in range(n_output):
                _w = np.pad(self.W[i], [(0, 0), (idx-diff, diff)],'constant')
                w_list.append(_w.reshape(1, -1))
                diff -= self.stride
            idx = diff = n_features - n_filters

        w_matrix = np.array(w_list).reshape(len(w_list), -1)
        dX = np.dot(dA.reshape(1, -1), w_matrix).reshape(self.x.shape)

        self.dW = np.dot(dA, self.x_matrix)
        self.dW = self.dW.reshape(self.W.shape)
        self.dB = np.array(dA.sum(axis=0))

        # # 更新
        # self = self.optimizer.update(self)

        return dX


    def _get_output_size(self):
        '''
        出力サイズの計算 n_output = (n_features + 2*padding - n_filters)/stride + 1
        '''

        n_features = self.x.shape[1]
        n_filters = self.W.shape[-1]
        return int((n_features + 2*self.padding - n_filters) / self.stride + 1)

In [37]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # shape(2, 4)で、（入力チャンネル数、特徴量数）である。
w = np.ones((3, 2, 3)) # 例の簡略化のため全て1とする。(出力チャンネル数、入力チャンネル数、フィルタサイズ)である。
b = np.array([1, 2, 3]) # （出力チャンネル数）

dA = np.array([16, 22, 17, 23, 18, 24]).reshape(3, 2)
cnn = Conv1d(w, b)
a = cnn._forward(x)
print("dA:\n", a)
b = cnn._backward(dA)
print("b:\n", b)

dA:
 [[16. 22.]
 [17. 23.]
 [18. 24.]]
b:
 [[ 51. 120. 120.  69.]
 [ 51. 120. 120.  69.]]


## 【問題8】学習と推定
---
これまで使ってきたニューラルネットワークの全結合層の一部をConv1dに置き換えてMNISTを学習・推定し、Accuracyを計算してください。

出力層だけは全結合層をそのまま使ってください。ただし、チャンネルが複数ある状態では全結合層への入力は行えません。その段階でのチャンネルは1になるようにするか、 平滑化 を行なってください。

画像に対しての1次元畳み込みは実用上は行わないことのため、精度は問いません。

In [38]:
class SimpleInitializer:
    """
    ガウス分布によるシンプルな初期化
    Parameters
    ----------
    sigma : float
      ガウス分布の標準偏差
    """
    def __init__(self, sigma):
        self.sigma = sigma
    def W(self, *shape):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W :
        """
        w = self.sigma * np.random.randn(*shape)
        return w

    def B(self, *shape):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :
        """
        b = self.sigma * np.random.randn(*shape)
        return b

class XavierInitializer:
    """
    Xavierによる初期化
    Parameters
    ----------
    """
    def __init__(self, sigma):
        _ = sigma

    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W :
        """
        sigma = 1.0 / np.sqrt(n_nodes1)
        w = sigma * np.random.randn(n_nodes1, n_nodes2)
        return w

    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :
        """
        b = np.random.randn(n_nodes2)
        return b

class HeInitializer:
    """
    Heによる初期化
    Parameters
    ----------
    """
    def __init__(self, sigma):
        _ = sigma

    def W(self, n_nodes1, n_nodes2):
        """
        重みの初期化
        Parameters
        ----------
        n_nodes1 : int
          前の層のノード数
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        W :
        """
        sigma = np.sqrt( 2.0 / n_nodes1)
        w = sigma * np.random.randn(n_nodes1, n_nodes2)
        return w

    def B(self, n_nodes2):
        """
        バイアスの初期化
        Parameters
        ----------
        n_nodes2 : int
          後の層のノード数
        Returns
        ----------
        B :
        """
        b = np.random.randn(n_nodes2)
        return b

# 最適化手法
class SGD:
    """
    確率的勾配降下法
    Parameters
    ----------
    lr : 学習率
    """
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """

        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB

        return layer

class AdaGrad:
    def __init__(self, lr):
        self.lr = lr

    def update(self, layer):
        """
        ある層の重みやバイアスの更新
        Parameters
        ----------
        layer : 更新前の層のインスタンス
        """
        DELTA = 1e-7
        layer.W_h += layer.dW * layer.dW
        layer.W -= self.lr * (1.0 / np.sqrt(layer.W_h + DELTA)) * layer.dW / layer.x.shape[0]

        layer.B_h += layer.dB * layer.dB
        layer.B -= self.lr * (1.0 / np.sqrt(layer.B_h + DELTA)) * layer.dB / layer.x.shape[0]

        return layer

def cross_entropy_error(y, Z3):
    '''
    交差エントロピー誤差の計算
    '''

    DELTA = 1e-7
    batch_size = y.shape[0]
    return -np.sum(y * np.log(Z3 + DELTA))/batch_size

# 活性化関数クラス化
class Softmax:
    """
    ソフトマックス関数
    Parameters
    ----------

    """
    def __init__(self):
        self.loss = None

    def forward(self, X):

        X = X.T
        y = np.exp(X) / np.sum(np.exp(X), axis=0)

        return y.T

    def backward(self, Z3, y):
        batch_size = y.shape[0]
        ret = (Z3 - y)/batch_size

        # lossの計算
        self.loss = cross_entropy_error(y, Z3)

        return ret

class ReLU:
    """
    ReLU関数
    Parameters
    ----------
    """
    def __init__(self):
        self.x = None

    def forward(self, X):

        self.x = X

        return np.maximum(0, X)

    def backward(self, X):

        return np.where(self.x > 0, X, 0)


class Tanh:
    """
    ハイボリックタンジェント関数
    Parameters
    ----------
    """
    def __init__(self):
        self.dA = None

    def forward(self, X):

        self.dA = np.tanh(X)
        return self.dA

    def backward(self, dZ):

        return dZ*(1 - np.tanh(self.dA)**2)

class FC:
    """
    ノード数n_nodes1からn_nodes2への全結合層
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    """
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.initializer = initializer

        self.W = self.initializer.W(self.n_nodes1, self.n_nodes2)
        self.B = self.initializer.B(self.n_nodes2)

        # 微分した重みとバイアス
        self.dW = None
        self.dB = None

        self.x = None

        # AdaGrad用のインスタンス変数
        self.W_h = 0
        self.B_h = 0

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """

        self.x = X  # backwardで使うため保持
        A = np.dot(X, self.W) + self.B
        return A

    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """

        dZ = np.dot(dA, self.W.T)
        self.dW = np.dot(self.x.T, dA)
        self.dB = np.sum(dA, axis=0)

        # 更新
        self = self.optimizer.update(self)

        return dZ



class ScratchConv1d:
    """
    畳み込み層
    Parameters
    ----------
    initializer : 初期化方法のインスタンス
    optimizer : 最適化手法のインスタンス
    n_filters : int
      フィルター数
    stride : int
      ストライド
    padding : int
      パディング
    in_channels : int
      入力チャンネル数
    n_out_channels : int
      出力チャンネル数
    """
    def __init__(self, initializer, optimizer, n_filters, stride=1, padding=1, in_channels=1, n_out_channels=1):
        self.optimizer = optimizer
        self.stride = stride
        self.padding = padding
        self.n_output = None
        self.in_channels = in_channels
        self.n_out_channels = n_out_channels
        self.n_filters = n_filters

        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.initializer = initializer

        # 重みとバイアスは全ての特徴量に共有で使用するため、FCのように各ノードごとに保持する必要はない
        self.W = self.initializer.W(n_out_channels, in_channels, n_filters)
        self.B = self.initializer.B(n_out_channels)

        # 微分した重みとバイアス
        self.dW = None
        self.dB = None

        self.x = None
        self.x_matrix = None
        self.dZ = None

        # AdaGrad用のインスタンス変数
        self.W_h = 0
        self.B_h = 0

    def forward(self, X):
        """
        フォワード
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_nodes1)
            入力
        Returns
        ----------
        A : 次の形のndarray, shape (batch_size, n_nodes2)
            出力
        """

        self.x = X  # backwardで使うため保持

        idx = 0
        n_filters = self.W.shape[2]
        self.n_output = self.get_output_size(self.x.shape[1], n_filters)
        n_channel = self.W.shape[0]
        A = np.zeros((n_channel, self.n_output))
        x_list = [] # backwardで使用する

        # 畳み込み演算用にXデータをフィルターと同じ形状に分割し、backwardで使用するため保持もする
        for _ in range(self.n_output):
            _X = X[:, idx:idx + n_filters]
            x_list.append(_X) #1次元配列に形状を変更（平滑化）
            idx += self.stride

        # backwardで使用するため保持する。
        self.x_matrix = np.array(x_list).reshape(self.n_output, -1)

        # 畳み込み演算
        for i in range(n_channel):
            for j, _X in enumerate(x_list):
                A[i][j] = (np.dot(_X.reshape(1, -1), self.W[i].reshape(1, -1).T)) + self.B[i]

        return A

    def backward(self, dA):
        """
        バックワード
        Parameters
        ----------
        dA : 次の形のndarray, shape (batch_size, n_nodes2)
            後ろから流れてきた勾配
        Returns
        ----------
        dZ : 次の形のndarray, shape (batch_size, n_nodes1)
            前に流す勾配
        """

        # n_row = len(self.x) - len(self.W) + self.stride
        n_features = self.x.shape[1]
        n_filters = self.W.shape[-1]
        #TODO self.n_outputに置き換えれるかも
        n_output = self.get_output_size(n_features, n_filters)
        n_channel, n_features = self.x.shape
        idx = diff = n_features - n_filters


        # 重みを平滑化にする
        w_flatten = self.W.reshape(-1, self.n_filters)

        w_list = []
        dX = None
        # for i in range(n_filters):
        for i in range(self.n_out_channels):
            for _ in range(n_output):
                if self.in_channels == 1:
                    _w = np.pad(w_flatten[i], [idx-diff, diff],'constant')
                    w_list.append(_w)
                else:
                    _w = np.pad(self.W[i], [(0, 0), (idx-diff, diff)],'constant')
                    w_list.append(_w.reshape(1, -1))
                diff -= self.stride
            idx = diff = n_features - n_filters

        if self.in_channels == 1:
            dX = np.sum(dA.reshape(-1,1) * np.array(w_list), axis=0)
            dX = dX.reshape(1, -1)
            self.dW = np.sum(self.x_matrix.T * dA, axis=1)
            self.dB = np.array(dA.sum())

        else:
            w_matrix = np.array(w_list).reshape(len(w_list), -1)
            dX = np.dot(dA.reshape(1, -1), w_matrix).reshape(self.x.shape)
            self.dW = np.dot(dA, self.x_matrix)
            self.dW = self.dW.reshape(self.W.shape)
            self.dB = np.array(dA.sum(axis=0))

        # 更新
        self.optimizer.update(self)

        return dX

    def get_output_size(self, n_features, n_filters):
        '''
        出力サイズの計算 n_output = (n_features + 2*padding - n_filters)/stride + 1
        '''
        return int((n_features + 2*self.padding - n_filters) / self.stride + 1)


### CNNのクラス

In [39]:
class Scratch1dCNNClassifier:
    def __init__(self, batch_size, n_features, n_nodes1, n_nodes2, n_output, lr, sigma=0.01, optimizer=AdaGrad, activation=Tanh, itializer=XavierInitializer):
        self.batch_size = batch_size
        self.n_features = n_features
        self.n_nodes1 = n_nodes1    # 1層目のノード数
        self.n_nodes2 = n_nodes2    # 2層目のノード数
        self.n_output = n_output    # 出力層のノード数
        self.lr = lr        # 学習率
        self.sigma = sigma  # ガウス分布の標準偏差

        self.itializer = itializer
        self.optimizer = optimizer
        self.activation = activation
        self.Conv1d = ScratchConv1d(initializer=SimpleInitializer(0.01), optimizer=self.optimizer(self.lr), n_filters=8, stride=1, padding=0, in_channels=1, n_out_channels=1)
        self.Conv1d.n_output = self.Conv1d.get_output_size(n_features, self.Conv1d.W.shape[2])
        self.activation1 = self.activation()
        self.FC2 = FC(self.Conv1d.n_output, self.n_nodes2, self.itializer(self.sigma), self.optimizer(self.lr))
        self.activation2 = self.activation()
        self.FC3 = FC(self.n_nodes2, self.n_output, self.itializer(self.sigma), self.optimizer(self.lr))
        self.activation3 = Softmax()

        self.loss_list = []

    def fit(self, X, y, X_val=None, y_val=None):

        Z3 = self._forward(X)
        self._backward(X, y, Z3)

        # 損失関数の値を保持
        self.loss_list.append(self.activation3.loss)


    def _forward(self, X):

        A1 = self.Conv1d.forward(X)
        Z1 = self.activation1.forward(A1)
        A2 = self.FC2.forward(Z1)
        Z2 = self.activation2.forward(A2)
        A3 = self.FC3.forward(Z2)
        Z3 = self.activation3.forward(A3)

        return Z3


    def _backward(self, X, y, Z3):

        dA3 = self.activation3.backward(Z3, y) # 交差エントロピー誤差とソフトマックスを合わせている
        dZ2 = self.FC3.backward(dA3)
        dA2 = self.activation2.backward(dZ2)
        dZ1 = self.FC2.backward(dA2)
        dA1 = self.activation1.backward(dZ1)
        dZ0 = self.Conv1d.backward(dA1) # dZ0は使用しない

    def predict(self, X):
        """
        ニューラルネットワーク分類器を使い推定する。
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_features)
            特徴量ベクトル
        Returns
        -------
        y_pred：次の形のndarray, shape (batch_size, n_output)
            推定結果（10個の確率の中で、最も高いインデックス＝各ラベル（0〜9））
        """

        y_pred_list = []
        for _, _x in enumerate(X):
            y_pred = self._forward(_x.reshape(1, -1))
            y_pred_list.append(y_pred.argmax(axis=1))

        return np.array(y_pred_list)


### ミニバッチ関数

In [40]:
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ
    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 20, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self._X = X[shuffle_index]
        self._y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self._X[p0:p1], self._y[p0:p1]
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self._X[p0:p1], self._y[p0:p1]

def get_mini_batch_data():
    from keras.datasets import mnist
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    X_train = X_train.reshape(-1, 784)
    X_test = X_test.reshape(-1, 784)

    X_train = X_train.astype(np.float)
    X_test = X_test.astype(np.float)
    X_train /= 255
    X_test /= 255

    from sklearn.model_selection import train_test_split

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

    # One-hotエンコーダー
    from sklearn.preprocessing import OneHotEncoder
    enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
    y_val_one_hot = enc.transform(y_val[:, np.newaxis])

    print(y_train_one_hot.shape)
    print(y_val_one_hot.shape)

    data = GetMiniBatch(X_train, y_train_one_hot, batch_size=20)


    return data


### データの準備

In [41]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# print("X:", X_train)

X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

# One-hotエンコーダー
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.2)

print(X_train.shape)
print(X_val.shape)

(48000, 784)
(12000, 784)


### 学習と推定

In [42]:
batch_size = 1 # バッチサイズ
n_features = 784 # 特徴量の数
n_nodes1 = 400 # 1層目のノード数
n_nodes2 = 200 # 2層目のノード数
n_output = 10 # 出力のクラス数（3層目のノード数）
lr = 0.01 # 学習率
epoch = 10 #エポック回数

cnn = Scratch1dCNNClassifier(batch_size, n_features, n_nodes1, n_nodes2, n_output, lr)

for p in range(epoch):

    get_mini_batch = GetMiniBatch(X_train, y_train, batch_size=batch_size, seed=p)

    for i in range(3000):

        mini_X_train, mini_y_train = get_mini_batch[i]
        # mini_X_val, mini_y_val = val_get_mini_batch[i]
        cnn.fit(mini_X_train, mini_y_train)


In [43]:
from sklearn.metrics import accuracy_score

y_pred = cnn.predict(X_test)
score = accuracy_score(y_test, y_pred)
print("score :", score)

score : 0.925
